# torch 搭建模型

torch搭建模型可以分为2类，第一是class的方法，另一个是快速搭建

- class
    - 直接初始化 Layer(torch里面都是继承至Module的)
    - 通过 Sequential 快速添加
    - 通过 Sequential 的 add 方法一个个组件添加
    - 通过 Sequential 添加 dict 来添加
- 快速搭建 （未找到可靠的依据---暂不示例，也不推荐）
    - 直接通过 Sequential 快速搭建

# 通过 继承 Module的方法进行搭建

In [31]:
import torch
import torch.nn.functional as F
from collections import OrderedDict
from torchsummary import summary

In [11]:
# 直接初始化 Layer(torch里面都是继承至Module的)

class Net_1(torch.nn.Module):
    def __init__(self):
        super(Net_1, self).__init__()
        # Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.conv_1 = torch.nn.Conv2d(3, 32, 3, 1, 1)
        self.pool_1 = torch.nn.MaxPool2d(2)
        self.dense_1 = torch.nn.Linear(32*3*3, 128) # 之所以是32*3*3是因为会经过一个池化层
        self.dense_2 = torch.nn.Linear(128, 10)
    
    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = self.pool_1(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.dense_1(x))
        x = self.dense_2(x)
        return x

summary(Net_1(), (3, 6, 6))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 32, 6, 6]             896
         MaxPool2d-2             [-1, 32, 3, 3]               0
            Linear-3                  [-1, 128]          36,992
            Linear-4                   [-1, 10]           1,290
Total params: 39,178
Trainable params: 39,178
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.15
Estimated Total Size (MB): 0.16
----------------------------------------------------------------


In [10]:
# 通过 Sequential 快速添加
class Net_2(torch.nn.Module):
    def __init__(self):
        super(Net_2, self).__init__()
        self.conv = torch.nn.Sequential(torch.nn.Conv2d(3, 32, 3, 1, 1),
                                        torch.nn.ReLU(), 
                                        torch.nn.MaxPool2d(2))
        self.dense = torch.nn.Sequential(torch.nn.Linear(32*3*3, 128),
                                         torch.nn.ReLU(),
                                         torch.nn.Linear(128, 10))
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.dense(x)
        return x
    
summary(Net_2(), (3, 6, 6))    

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 32, 6, 6]             896
              ReLU-2             [-1, 32, 6, 6]               0
         MaxPool2d-3             [-1, 32, 3, 3]               0
            Linear-4                  [-1, 128]          36,992
              ReLU-5                  [-1, 128]               0
            Linear-6                   [-1, 10]           1,290
Total params: 39,178
Trainable params: 39,178
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.15
Estimated Total Size (MB): 0.17
----------------------------------------------------------------


In [30]:
# 通过 Sequential 的 add 方法一个个组件添加
class Net_3(torch.nn.Module):
    def __init__(self):
        super(Net_3, self).__init__()
        self.conv = torch.nn.Sequential()
        self.conv.add_module('conv_1', torch.nn.Conv2d(3, 32, 3, 1, 1))
        self.conv.add_module('relu_1', torch.nn.ReLU())
        self.conv.add_module('pool_1', torch.nn.MaxPool2d(2))
        self.dense = torch.nn.Sequential()
        self.dense.add_module('dense_1', torch.nn.Linear(32*3*3, 128))
        self.dense.add_module('relu_2', torch.nn.ReLU())
        self.dense.add_module('dense_2', torch.nn.Linear(128, 10))
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.dense(x)
        return x
    
summary(Net_3(), (3, 6, 6))  
print(Net_3())

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 32, 6, 6]             896
              ReLU-2             [-1, 32, 6, 6]               0
         MaxPool2d-3             [-1, 32, 3, 3]               0
            Linear-4                  [-1, 128]          36,992
              ReLU-5                  [-1, 128]               0
            Linear-6                   [-1, 10]           1,290
Total params: 39,178
Trainable params: 39,178
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.15
Estimated Total Size (MB): 0.17
----------------------------------------------------------------
Net_3(
  (conv): Sequential(
    (conv_1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_1): ReLU()
    (pool_1): MaxPool2d(kernel_size=2, stri

In [32]:
# 通过 Sequential 添加 dict 来添加

class Net_4(torch.nn.Module):
    def __init__(self):
        super(Net_4, self).__init__()
        self.conv = torch.nn.Sequential(
            OrderedDict([
                ("conv_1", torch.nn.Conv2d(3, 32, 3, 1, 1)),
                ("rule_1", torch.nn.ReLU()),
                ("pool", torch.nn.MaxPool2d(2))
            ])
        )
        self.dense = torch.nn.Sequential(
            OrderedDict([
                ("dense_1", torch.nn.Linear(32*3*3, 128)),
                ("rule_2", torch.nn.ReLU()),
                ("desnse_2", torch.nn.Linear(128, 10))
            ])
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.dense(x)
        return x

summary(Net_4(), (3, 6, 6))  
print(Net_4())

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 32, 6, 6]             896
              ReLU-2             [-1, 32, 6, 6]               0
         MaxPool2d-3             [-1, 32, 3, 3]               0
            Linear-4                  [-1, 128]          36,992
              ReLU-5                  [-1, 128]               0
            Linear-6                   [-1, 10]           1,290
Total params: 39,178
Trainable params: 39,178
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.15
Estimated Total Size (MB): 0.17
----------------------------------------------------------------
Net_4(
  (conv): Sequential(
    (conv_1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (rule_1): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride